### catalogo de datos madrid

In [1]:
#parse dcat xml
import xml.etree.ElementTree as ET
import urllib.request

ns = {
    'foaf': "http://xmlns.com/foaf/0.1/",
    'skos': "http://datos.gob.es/kos/sector-publico/sector/",
    'dcat': "http://www.w3.org/ns/dcat#",
    'rdfs': "http://www.w3.org/2000/01/rdf-schema#",
    'rdf': "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    'dct': "http://purl.org/dc/terms/",
    'dc': "http://purl.org/dc/elements/1.1/",
    'time': "http://www.w3.org/2006/time#",
    'xsd': "http://www.w3.org/2001/XMLSchema#",
    'j.0': "http://www.w3.org/1999/xhtml/vocab#",
    'j.1': "http://a9.com/-/spec/opensearch/1.1/",
    'api': "http://purl.org/linked-data/api/vocab#",
}

with urllib.request.urlopen('https://datos.madrid.es/egob/catalogo.rdf') as f:
    tree = ET.parse(f)
    root = tree.getroot()

In [ ]:
from datetime import datetime
ds_list = [{'indentifier': dataset.find('dct:identifier', ns).text,
            'dataset_title': dataset.find('dct:title', ns).text
            , 'theme': dataset.find('dcat:theme', ns).attrib.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource')
            , 'keywords': dataset.find('dcat:keyword', ns).text
            , 'issued':  datetime.strptime(dataset.find('dct:issued', ns).text, '%Y-%m-%dT%H:%M:%S')
            , 'modified': datetime.strptime(dataset.find('dct:modified', ns).text, '%Y-%m-%dT%H:%M:%S')
            , 'frequency': dataset.find('dct:accrualPeriodicity/dct:Frequency/rdfs:label', ns).text if  dataset.find('dct:accrualPeriodicity/dct:Frequency/rdfs:label', ns) is not None else None
            , 'distribution_title': distribution.find('dct:title', ns).text if distribution.find('dct:title', ns) is not None else None
            , 'mediaType': distribution.find('dcat:mediaType', ns).text
            , 'accessURL': distribution.find('dcat:accessURL', ns).text
            }
           for dataset
           in root.findall('./dcat:Catalog/dcat:dataset/dcat:Dataset', ns)
           for distribution
           in dataset.findall('dcat:distribution/dcat:Distribution', ns)

           ]

len(ds_list)

In [ ]:
import requests
requests.packages.urllib3.disable_warnings() 
import csv

with open('../data/catalogo_validacion4.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    for i, d in enumerate(ds_list[4947:]):
        #print(d['indentifier'], d['dataset_title'], d['distribution_title'], d['mediaType'], d['accessURL'] ,sep = '; ')
        try:
            x = requests.head(d['accessURL'], allow_redirects=True, verify=False)
            #print('\t', x.status_code)
            #print('\t', x.headers)
            #print('\t', x.headers['Date'], x.headers['Last-Modified'], x.headers['Content-Type'], x.headers['Content-Length'],sep = '; ' )
            print(i, d['dataset_title'], d['distribution_title'], x.status_code, x.headers['Content-Type'], x.headers['Content-Length'],  sep = '; ')
            csv_writer.writerow([d['indentifier'], d['dataset_title'], d['distribution_title'], d['mediaType'], d['accessURL'] ,x.status_code,x.url, x.headers['Date'], x.headers['Last-Modified'], x.headers['Content-Type'], x.headers['Content-Length'], x.elapsed])
        except:
            print(i, d['indentifier'], d['dataset_title'], d['distribution_title'], d['mediaType'], d['accessURL'] ,x.status_code, 'ERROR', x.elapsed, sep='; ') 
            csv_writer.writerow([d['indentifier'], d['dataset_title'], d['distribution_title'], d['mediaType'], d['accessURL'] ,x.status_code,x.url,x.elapsed, 'ERROR']) 

    



In [ ]:
import requests

x = requests.head('https://datos.madrid.es/egob/catalogo/300724-12621499-Seccionado-censal-mapas.xls', allow_redirects=True, verify=False)

print(x.status_code)
print(x.url)
print(x.headers)